In [ ]:
# Guarded Setup
DRY_RUN = True
from notebooks._utils.common import *
CLI_OK = shell_available('forensic-cli')
LAB_ID = '20_network_to_timeline_solution'
LAB_ROOT = lab_root(LAB_ID)
print(f'CLI available: {CLI_OK}')
print(f'Artifacts root: {LAB_ROOT}')


# Solution · Network to Timeline

This walkthrough mirrors the lab workflow using the same deterministic data.

In [ ]:
from pathlib import Path
import json
import csv
from datetime import datetime
from forensic.core.framework import ForensicFramework
from forensic.modules.analysis.network import NetworkAnalysisModule
from forensic.modules.analysis.timeline import TimelineModule

WORKSPACE = LAB_ROOT / 'workspace'
CASE_ID = 'CASE_LAB20_SOLUTION'
framework = ForensicFramework(workspace=WORKSPACE)
registered = set(framework.list_modules())
if 'network' not in registered:
    framework.register_module('network', NetworkAnalysisModule)
if 'timeline' not in registered:
    framework.register_module('timeline', TimelineModule)
try:
    case = framework.load_case(CASE_ID)
except ValueError:
    case = framework.create_case(
        name='Lab 20 Solution Case',
        description='Reference implementation for the exercise.',
        investigator='Notebook Analyst',
        case_id=CASE_ID,
    )
case.case_id


In [ ]:
pcap_payload = {
    'flows': [
        {
            'src': '10.0.0.5',
            'dst': '192.168.1.10',
            'src_port': 52344,
            'dst_port': 80,
            'protocol': 'TCP',
            'packets': 6,
            'bytes': 512,
            'start_ts': '2024-03-01T12:00:00Z',
            'end_ts': '2024-03-01T12:00:05Z',
        },
        {
            'src': '10.0.0.5',
            'dst': '8.8.8.8',
            'src_port': 55344,
            'dst_port': 53,
            'protocol': 'UDP',
            'packets': 2,
            'bytes': 128,
            'start_ts': '2024-03-01T11:59:50Z',
            'end_ts': '2024-03-01T11:59:51Z',
        },
        {
            'src': '192.168.1.10',
            'dst': '10.0.0.5',
            'src_port': 80,
            'dst_port': 52344,
            'protocol': 'TCP',
            'packets': 4,
            'bytes': 1024,
            'start_ts': '2024-03-01T12:00:01Z',
            'end_ts': '2024-03-01T12:00:04Z',
        },
    ],
    'dns': {
        'queries': [
            {
                'timestamp': '2024-03-01T11:59:50Z',
                'query': 'updates.example.com',
                'query_type': 1,
                'src': '10.0.0.5',
                'dst': '8.8.8.8',
                'src_port': 55344,
                'dst_port': 53,
                'protocol': 'UDP',
                'heuristics': {'high_entropy': False, 'long_domain': False},
            },
            {
                'timestamp': '2024-03-01T11:59:55Z',
                'query': 'payload.control-node.example',
                'query_type': 28,
                'src': '10.0.0.5',
                'dst': '8.8.4.4',
                'src_port': 55345,
                'dst_port': 53,
                'protocol': 'UDP',
                'heuristics': {'high_entropy': False, 'long_domain': True},
            },
        ]
    },
    'http': {
        'requests': [
            {
                'timestamp': '2024-03-01T12:00:02Z',
                'method': 'GET',
                'host': 'intranet.local',
                'uri': '/status',
                'user_agent': 'NotebookLab/1.0',
                'src': '10.0.0.5',
                'dst': '192.168.1.10',
                'src_port': 52344,
                'dst_port': 80,
                'protocol': 'TCP',
                'indicators': {'encoded_uri': False, 'suspicious_user_agent': False},
            },
            {
                'timestamp': '2024-03-01T12:00:03Z',
                'method': 'POST',
                'host': 'collector.example.net',
                'uri': '/beacon',
                'user_agent': 'curl/7.88.1',
                'src': '10.0.0.5',
                'dst': '198.51.100.23',
                'src_port': 52345,
                'dst_port': 443,
                'protocol': 'TCP',
                'indicators': {'encoded_uri': False, 'suspicious_user_agent': True},
            },
        ]
    },
}
pcap_input_path = LAB_ROOT / 'inputs' / 'pcap_synthetic.json'
pcap_input_path.parent.mkdir(parents=True, exist_ok=True)
json_dump_sorted(pcap_payload, pcap_input_path)
pcap_input_path


In [ ]:
network_result = framework.execute_module(
    'network',
    params={'pcap_json': str(pcap_input_path)},
)
network_output = Path(network_result.output_path)
network_payload = json.loads(network_output.read_text(encoding='utf-8'))
NETWORK_EXPORT = LAB_ROOT / 'network_outputs'
NETWORK_EXPORT.mkdir(parents=True, exist_ok=True)
flows_path = json_dump_sorted(network_payload.get('flows', []), NETWORK_EXPORT / 'flows.json')
dns_path = json_dump_sorted(network_payload.get('dns', {}), NETWORK_EXPORT / 'dns.json')
http_path = json_dump_sorted(network_payload.get('http', {}), NETWORK_EXPORT / 'http.json')
{'status': network_result.status, 'output': str(network_output)}


In [ ]:
network_output_dir = network_output.parent
timeline_result = framework.execute_module(
    'timeline',
    params={
        'source': str(network_output_dir),
        'format': 'csv',
        'type': 'simple',
    },
)
TIMELINE_EXPORT = LAB_ROOT / 'timeline_outputs'
TIMELINE_EXPORT.mkdir(parents=True, exist_ok=True)
timeline_csv = Path(timeline_result.metadata['unified_timeline']['csv_file'])
timeline_json = Path(timeline_result.metadata['unified_timeline']['json_file'])
with timeline_csv.open(encoding='utf-8') as handle:
    reader = csv.DictReader(handle)
    timeline_rows = list(reader)
json_dump_sorted(timeline_rows, TIMELINE_EXPORT / 'timeline_events.json')
csv_write_rows_sorted(timeline_rows, TIMELINE_EXPORT / 'timeline_events.csv', header=reader.fieldnames)
{'timeline_events': len(timeline_rows)}


In [ ]:
from pprint import pprint
from datetime import datetime

def _parse_timestamp(value: str) -> datetime:
    return datetime.fromisoformat(value.replace('Z', '+00:00'))

pprint(timeline_rows[:4])
